# Customer Segmentation using K-Means: Online Retail Dataset

En este caso de estudio se muestran los pasos necesarios para identificar grupos de clientes utilizando el Online Retail Dataset (https://www.kaggle.com/puneetbhaya/online-retail). 

En este notebook se reproducen (con pequeñas modificaciones) los pasos dados en este Notebook de Kaggle (https://www.kaggle.com/mgmarques/customer-segmentation-and-market-basket-analysis).

Comenzamos importando las librerías necesarias y cargando el fichero que contiene los datos de entrada.

In [ ]:
import pandas as pd
import datetime
import math
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
import matplotlib.cm as cm

cs_df = pd.read_excel(io=r'online-retail.xlsx')

## Exploratory Data Analysis (EDA)

Es una buena práctica comenzar con un análisis exploratorio de datos para conocer el dataset al que nos enfrentamos e identificar posibles anomalías o situaciones que haya que tener en cuenta para el análisis (valores perdidos, variables que sea necesario escalar, presencia de outliers, etc.).

In [ ]:
print(cs_df.head())

cs_df.describe()

Definimos una función (inspirada en `str` de R) que muestra las características generales de cada columna del dataset: número de valores, número de valores nulos, número de valores distintos, porcentaje de valores perdidos, skewness (https://en.wikipedia.org/wiki/Skewness) y kurtosis (https://en.wikipedia.org/wiki/Kurtosis).

In [ ]:
def rstr(df): 
    obs = df.shape[0]
    types = df.dtypes
    counts = df.apply(lambda x: x.count())
    uniques = df.apply(lambda x: [x.unique()])
    nulls = df.apply(lambda x: x.isnull().sum())
    distincts = df.apply(lambda x: x.unique().shape[0])
    missing_ration = (df.isnull().sum()/ obs) * 100
    skewness = df.skew()
    kurtosis = df.kurt()
    skewness = df.skew()
    kurtosis = df.kurt() 
    
    str = pd.DataFrame({
        'types': types, 
        'counts': counts, 
        'nulls': nulls, 
        'distincts': distincts, 
        'missing_ration': missing_ration,
        'skewness': skewness,
        'kurtosis': kurtosis
    })

    return str

details = rstr(cs_df)
display(details.sort_values(by='missing_ration', ascending=False))

Podemos observar que las variables `Quantity` y `UnitPrice` tienen valores negativos, lo cual puede significar que hay transacciones correspondientes a devoluciones. Como queremos segmentar clientes, es importante limpiar el dataset para eliminar las filas correspondientes primero.

Vamos a analizar el dataset para ver si existen filas en las que ambos valores sean negativos o si uno es negativo y el otro cero.

In [ ]:
print('Check if we had negative quantity and prices at same register:',
     'No' if cs_df[(cs_df.Quantity<0) & (cs_df.UnitPrice<0)].shape[0] == 0 else 'Yes', '\n')
print('Check how many register we have where quantity is negative',
      'and prices is 0 or vice-versa:',
      cs_df[(cs_df.Quantity<=0) & (cs_df.UnitPrice<=0)].shape[0])
print('\nWhat is the customer ID of the registers above:',
      cs_df.loc[(cs_df.Quantity<=0) & (cs_df.UnitPrice<=0), 
                ['CustomerID']].CustomerID.unique())
print('\n% Negative Quantity: {:3.2%}'.format(cs_df[(cs_df.Quantity<0)].shape[0]/cs_df.shape[0]))
print('\nAll register with negative quantity has Invoice start with:', 
      cs_df.loc[(cs_df.Quantity<0) & ~(cs_df.CustomerID.isnull()), 'InvoiceNo'].apply(lambda x: x[0]).unique())
print('\nSee an example of negative quantity and others related records:')
display(cs_df[(cs_df.CustomerID==12472) & (cs_df.StockCode==22244)])

No existen registros en que ambos valores sean negativos. Veamos ejemplo de registros con un precio negativo o cero:

In [ ]:
print('Check register with UnitPrice negative:', cs_df[(cs_df.UnitPrice<0)].shape[0])
display(cs_df[(cs_df.UnitPrice<0)])

print("Sales records with Customer ID and zero in Unit Price:",cs_df[(cs_df.UnitPrice==0)  & ~(cs_df.CustomerID.isnull())].shape[0])
display(cs_df[(cs_df.UnitPrice==0)  & ~(cs_df.CustomerID.isnull())])

print("Sales records without Customer ID:",cs_df[cs_df.CustomerID.isnull()].shape[0])

Como podemos ver, no hay registros con la cantidad y el precio negativos al mismo tiempo, pero sí que hay 1336 registros donde uno de ellos lo es y el otro es cero. Para estos registros, además, no existe un ID de cliente. Además, hay 135 080 sin ID de cliente en total. Podemos concluir que es necesario eliminar estos registros antes de proceder con el análisis.

In [ ]:
# Remove register without CustomerID
cs_df = cs_df[~(cs_df.CustomerID.isnull())]

# Remove negative or return transactions
cs_df = cs_df[~(cs_df.Quantity<0)]
cs_df = cs_df[cs_df.UnitPrice>0]

In [ ]:
cs_df.describe()

In [ ]:
### details = rstr(cs_df)
display(details.sort_values(by='distincts', ascending=False))

Añadimos una columna `amount` para guardar la cantidad total gastada y convertimos las fechas:

In [ ]:
cs_df.InvoiceDate = pd.to_datetime(cs_df.InvoiceDate)
cs_df['amount'] = cs_df.Quantity*cs_df.UnitPrice
cs_df.CustomerID = cs_df.CustomerID.astype('Int64')

details = rstr(cs_df)
display(details.sort_values(by='distincts', ascending=False))

## Variables del modelo RFM

### Recency

Para crear la variable `recency` debemos establecer una fecha de referencia para el análisis. Normalmente seleccionamos la fecha de la transacción más antigua más un día y luego construimos la variable `recency` contando el número de días de cada cliente desde su última compra.

In [ ]:
refrence_date = cs_df.InvoiceDate.max() + datetime.timedelta(days = 1)
print('Reference Date:', refrence_date)
cs_df['days_since_last_purchase'] = (refrence_date - cs_df.InvoiceDate).astype('timedelta64[D]')
customer_history_df =  cs_df[['CustomerID', 'days_since_last_purchase']].groupby("CustomerID").min().reset_index()
customer_history_df.rename(columns={'days_since_last_purchase':'recency'}, inplace=True)
customer_history_df.describe().transpose()

Hacemos un histograma con la distribución de esta nueva variable y podemos observar que tiene uan asimetría importante (`skewed right`) y un pico casi al final. En estos casos se puede transformar la variable empleando el logaritmo (u otra transformación) que elimine esta asimetría y haga que la distribución se asemeje más a una normal.

In [ ]:
plt.hist(customer_history_df['recency'], color = 'blue', edgecolor = 'black', bins = 90)

### Frequency

Para crear la variable `frequency` simplemente contamos el número de transacciones de cada cliente:

In [ ]:
customer_freq = (cs_df[['CustomerID', 'InvoiceNo']].groupby(["CustomerID", 'InvoiceNo']).count().reset_index()).\
                groupby(["CustomerID"]).count().reset_index()
customer_freq.rename(columns={'InvoiceNo':'frequency'},inplace=True)
customer_history_df = customer_history_df.merge(customer_freq)

Al igual que con la variable anterior, si hacemos un histograma con la distribución de esta nueva variable y podemos observar que tiene uan asimetría bastante marcada (`skewed right`).

In [ ]:
plt.hist(customer_history_df['frequency'], color = 'blue', edgecolor = 'black', bins = 90)

### Frequency

Para crear la variable `monetary` simplemente sumamos el valor de `amount` para cada cliente:

In [ ]:
customer_monetary_val = cs_df[['CustomerID', 'amount']].groupby("CustomerID").sum().reset_index()
customer_history_df = customer_history_df.merge(customer_monetary_val)

Al igual que con las variables anteriores, si hacemos un histograma con la distribución de esta nueva variable y podemos observar que tiene uan asimetría bastante marcada (`skewed right`).

In [ ]:
plt.hist(customer_history_df['amount'], color = 'blue', edgecolor = 'black', bins = 90)

Mostramos el nuevo DataFrame con el modelo RFM:

In [ ]:
display(customer_history_df.head())

display(customer_history_df.describe())

details = rstr(customer_history_df)
display(details)

## Preprocesado de datos

Ahora que tenemos la matriz con el modelo RFM, vamos a preprocesarla. Por un lado, escalaremos los datos para que todos estén en la misma escala, pues ya hemos visto que K-Means es sensible a variables en distinta escala y en este caso queremos que tengan todas la misma importancia. Además, las variables tienen una distribución asimétrica por la derecha y algunas, especialmente `amount`, toman un rango muy grande de valores. Los redistribuiremos haciendo la transformación logarítmica antes que el escalado.

**Importante**: el escalado es reversible, de manera que podemos transformar los centroides del clustering y que sean interpretables en términos del dataset original.

In [ ]:
customer_history_df['recency_log'] = customer_history_df['recency'].apply(math.log)
customer_history_df['frequency_log'] = customer_history_df['frequency'].apply(math.log)
customer_history_df['amount_log'] = customer_history_df['amount'].apply(math.log)
feature_vector = ['amount_log', 'recency_log','frequency_log']

X_subset = customer_history_df[feature_vector]
scaler = preprocessing.StandardScaler().fit(X_subset)

X_scaled = scaler.transform(X_subset)

X_scaled_df = pd.DataFrame(X_scaled, columns=X_subset.columns)

display(X_scaled_df.describe().T)

details = rstr(X_scaled_df)
display(details)

## K-Means clustering


Comenzamos aplicando el método Elbow y calculando los silhouette scores para tener una idea del mejor valor de k.

In [ ]:
from sklearn.cluster import KMeans
from sklearn import metrics

k_range = range(2, 20)

kmeans = [KMeans(n_clusters=i) for i in k_range]
inertia = [kmeans[i].fit(X_scaled).inertia_ for i in range(len(kmeans))]
labels_k = [kmeans[i].predict(X_scaled) for i in range(len(kmeans))]
silhouettes = [metrics.silhouette_score(X_scaled, labels_k[i], metric = 'euclidean') for i in range(len(kmeans))]

plt.xlabel('k')
plt.ylabel('inertia')
plt.plot(k_range, inertia, linestyle='--', marker='o', color='black')
plt.show()

silhouettes_df = pd.DataFrame({'k': k_range, 'silhouette': silhouettes}).sort_values('silhouette')

display(silhouettes_df)

Parece que a partir de 6-7 clusters se frena la reducción de la inercia pero no existe un candidato claro (no se ve tal codo). El menor silhouette se obtiene en k = 7, así que probaremos los valores 3, 5 y 7 (7 porque parece el mejor y los otros dos porque producirán menos grupos que podrían ser más fáciles de interpretar).

En este artículo se puede ver una descripción más completa de los Silhouette plots que crearemos a continuación: https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html Estos gráficos muestran las muestras agrupadas y una línea con el valor medio del silhouette. Un k dado sería una mala elección si existen clusters por debajo del silhouette medio (línea roja discontinua) y/o con grandes fluctuaciones en el silhouette score de sus muestras.

In [ ]:
k_values = [3, 5, 7]

cluster_centers = dict()

for n_clusters in k_values:
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
    fig.set_size_inches(25, 7)
    ax1.set_xlim([-0.1, 1])
    ax1.set_ylim([0, len(X_scaled) + (n_clusters + 1) * 10])

    clusterer = KMeans(n_clusters=n_clusters, init='k-means++', n_init=10,max_iter=300, tol=1e-04, random_state=101)
    cluster_labels = clusterer.fit_predict(X_scaled)

    silhouette_avg = metrics.silhouette_score(X = X_scaled, labels = cluster_labels)
    cluster_centers.update({n_clusters :{'cluster_center':clusterer.cluster_centers_,
                                         'silhouette_score':silhouette_avg,
                                         'labels':cluster_labels}
                           })

    sample_silhouette_values = metrics.silhouette_samples(X = X_scaled, labels = cluster_labels)
    y_lower = 10
    for i in range(n_clusters):
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.Spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")
    ax1.set_yticks([])
    ax1.set_xticks([-0.1, 0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1])
    colors = cm.Spectral(cluster_labels.astype(float) / n_clusters)
    
    centers = clusterer.cluster_centers_
    y = 0
    x = 1
    ax2.scatter(X_scaled[:, x], X_scaled[:, y], marker='.', s=30, lw=0, alpha=0.7, c=colors, edgecolor='k')   
    ax2.scatter(centers[:, x], centers[:, y], marker='o', c="white", alpha=1, s=200, edgecolor='k')
    for i, c in enumerate(centers):
        ax2.scatter(c[x], c[y], marker='$%d$' % i, alpha=1, s=50, edgecolor='k')
    ax2.set_title("{} Clustered data".format(n_clusters))
    ax2.set_xlabel(feature_vector[x])
    ax2.set_ylabel(feature_vector[y])

    x = 2
    ax3.scatter(X_scaled[:, x], X_scaled[:, y], marker='.', s=30, lw=0, alpha=0.7, c=colors, edgecolor='k')   
    ax3.scatter(centers[:, x], centers[:, y], marker='o', c="white", alpha=1, s=200, edgecolor='k')
    for i, c in enumerate(centers):
        ax3.scatter(c[x], c[y], marker='$%d$' % i, alpha=1, s=50, edgecolor='k')
    ax3.set_title("Silhouette score: {:1.2f}".format(cluster_centers[n_clusters]['silhouette_score']))
    ax3.set_xlabel(feature_vector[x])
    ax3.set_ylabel(feature_vector[y])
    
    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')
    plt.show()

## Anállisis de los clusters obtenidos

Finalmente, convertimos los centroides de los clusters a los valores originales y procedemos a analizar dichos clusters.

In [ ]:
features = ['amount',  'recency',  'frequency']
for i in k_values:
    print("for {} clusters the silhouette score is {:1.2f}".format(i, cluster_centers[i]['silhouette_score']))
    print("Centers of each cluster:")
    cent_transformed = scaler.inverse_transform(cluster_centers[i]['cluster_center'])
    print(pd.DataFrame(np.exp(cent_transformed),columns=features))
    print('-'*50)

Por ejemplo, en el caso de k = 3, identificamos:
- Que el cluster 2 incluye a clientes que compran con bastante frecuencia y gastan una cantidad considerable.
- Que los clusters 0 y 1 incluyen a dos segmentos de clientes que compran con poca frecuencia y gastan menos.

En el kaso de k = 7 se identifica un grupo muy interesante, el que corresponde al grupo 4: clientes que gastan una cantidad considerable, con mucha frecuencia y cuya última transacción es muy reciente.

Para terminar el análisis, vamos a añadir las etiquetas generadas en cada agrupamiento al DataFrame que teníamos y examinar los grupos.

In [ ]:
customer_history_df['clusters_3'] = cluster_centers[3]['labels'] 
customer_history_df['clusters_5'] = cluster_centers[5]['labels']
customer_history_df['clusters_7'] = cluster_centers[7]['labels']
display(customer_history_df.head())

fig = plt.figure(figsize=(20,7))
f1 = fig.add_subplot(131)
market = customer_history_df.clusters_3.value_counts()
g = plt.pie(market, labels=market.index, autopct='%1.1f%%', shadow=True, startangle=90)
plt.title('3 Clusters')
f1 = fig.add_subplot(132)
market = customer_history_df.clusters_5.value_counts()
g = plt.pie(market, labels=market.index, autopct='%1.1f%%', shadow=True, startangle=90)
plt.title('5 Clusters')
f1 = fig.add_subplot(133)
market = customer_history_df.clusters_7.value_counts()
g = plt.pie(market, labels=market.index, autopct='%1.1f%%', shadow=True, startangle=90)
plt.title('7 Clusters')
plt.show()

Podemos hacer diagramas de cajas de cada variable para ver cómo se distribuyen, además de imprimir algunas estadísticas de cada agrupamiento:

In [ ]:
clustering = 'clusters_3'

customer_history_df.boxplot(column=['frequency'], by=clustering)
customer_history_df.boxplot(column=['recency'], by=clustering)
customer_history_df.boxplot(column=['amount'], by=clustering)

In [ ]:
display(customer_history_df.groupby([clustering]).mean())
display(customer_history_df.groupby([clustering]).std())
display(customer_history_df.groupby([clustering])['amount'].describe())